In [ ]:
%reload_ext autoreload
%autoreload 2

# Sandbox

In [ ]:
import settings
from combolists import ComboList
from accesslevel import AccessLevel
from dotenv import load_dotenv
import warnings
from card import Card
from colaborador import Colaborador
from paciente import Paciente
from time import sleep
from helperfunctions import helper
from database import DataBase
from acompanhante import Acompanhante
from estudante import Estudante
import datetime
import requests
import json

warnings.filterwarnings("ignore")
load_dotenv()
settings.init()
settings.combofields = ComboList.getComboLists()
settings.accesslevels = AccessLevel.getAccessLevels()

In [ ]:
import csv

# Open the file ativos.csv and read it's content as a dictionary
with open('ativos.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=';')
    # Loop through the rows of the file
    
    for row in csv_reader:
        cardholder = requests.get(
            f'{settings.baseUrl}/cardholders/{row["chid"]}',
            verify=False,
        ).json()
        cardholder["CHState"] = 1

        if row["Last"] == 'NULL':
            result = requests.put(
                f'{settings.baseUrl}/cardholders',
                verify=False,
                headers={"Content-Type": "application/json"},
                data=json.dumps(cardholder, default=str),
            )
            sleep(0.5)

In [ ]:
# log = {}
# for paciente in pacientes_diarios:
#     log = Paciente.ativarPaciente(paciente["prontuario"])

In [ ]:
# Paciente.ativarPaciente(10201527)
# acompanhantes = requests.get(url=f"{settings.baseUrl}/cardholders?chType=7&includeTables=Cards&limit=1000000", verify=False)
# for acompanhante in acompanhantes.json():
#     Paciente.desativarPaciente(acompanhante["IdNumber"])
    

# pacientes_diarios = Paciente.getLiberacoesDiarias()

# for paciente in pacientes_diarios:
#   Paciente.ativarPaciente(paciente["prontuario"])

# pacientes = Paciente.getLiberacoes(settings.lastUpdated)
# for paciente in pacientes:
#   if paciente["CHState"] == 1:
#     Paciente.desativarPaciente(paciente["IdNumber"])
#   elif paciente["CHState"] == 0:
#     Paciente.ativarPaciente(paciente["IdNumber"])

# helper.updateTime(currentDate)

# Paciente.ativarPaciente(10000016)
# paciente_invenzi = Paciente.getPacienteByIdNumber(10000016)
# print(json.dumps(paciente_invenzi[0]["Cards"][0], indent=2))

## Listar Usuários por Vínculo usando combolists
- listar todos os usuários
